In [16]:
# importing libraries
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random


In [17]:
import nltk
import json
import pickle



nltk.download('punkt')
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer




[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [18]:
# tokenizing and lematizing
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
# lemmatize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
#sorts unique lemmatized words
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('texts.pkl','wb'))
pickle.dump(classes,open('labels.pkl','wb'))
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)

283 documents
129 classes ['What are the types of depression?', 'about', 'afternoon', 'anxious', 'ask', 'at what age does anxiety peak?', 'can lack of sleep make you feel sad?', 'can low blood sugar cause suicidal thoughts?', 'casual', 'creation', 'death', 'default', 'depressed', 'do we control our thoughts?', 'does oversleeping cause depression?', 'done', 'evening', 'fact-1', 'fact-10', 'fact-11', 'fact-12', 'fact-13', 'fact-14', 'fact-15', 'fact-16', 'fact-17', 'fact-18', 'fact-19', 'fact-2', 'fact-20', 'fact-21', 'fact-22', 'fact-23', 'fact-24', 'fact-25', 'fact-26', 'fact-27', 'fact-28', 'fact-29', 'fact-3', 'fact-30', 'fact-31', 'fact-32', 'fact-5', 'fact-6', 'fact-7', 'fact-8', 'fact-9', 'friends', 'goodbye', 'greeting', 'happy', 'hate-me', 'hate-you', 'help', 'how can we reduce anxiety?', 'how does depression affect the world?', 'how long can anxiety last?', 'how many thoughts a day do we have?', 'i am a victim of bullying', 'i am afraid i will fail again', 'i am afraid to file 

In [19]:
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])


In [20]:
import numpy as np
import random
random.shuffle(training)
training = np.array(training, dtype="object")
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


Training data created


In [6]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [21]:
from tensorflow.keras.optimizers import SGD

sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [22]:


#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('model.h5', hist)
print("model created")

Epoch 1/200
57/57 [==============================] - 1s 2ms/step - loss: 0.7186 - accuracy: 0.7880
Epoch 2/200
57/57 [==============================] - 0s 2ms/step - loss: 0.8150 - accuracy: 0.7562
Epoch 3/200
57/57 [==============================] - 0s 2ms/step - loss: 0.7771 - accuracy: 0.8057
Epoch 4/200
57/57 [==============================] - 0s 2ms/step - loss: 0.7813 - accuracy: 0.7703
Epoch 5/200
57/57 [==============================] - 0s 2ms/step - loss: 0.7478 - accuracy: 0.7809
Epoch 6/200
57/57 [==============================] - 0s 2ms/step - loss: 0.6990 - accuracy: 0.7809
Epoch 7/200
57/57 [==============================] - 0s 2ms/step - loss: 0.8203 - accuracy: 0.7739
Epoch 8/200
57/57 [==============================] - 0s 2ms/step - loss: 0.6320 - accuracy: 0.8339
Epoch 9/200
57/57 [==============================] - 0s 2ms/step - loss: 0.7881 - accuracy: 0.7739
Epoch 10/200
57/57 [==============================] - 0s 2ms/step - loss: 0.6703 - accuracy: 0.8163
Epoch 11/

c:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [23]:
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer

In [26]:
import random
import json
import nltk
from nltk.stem import WordNetLemmatizer
import numpy as np
from flask import Flask, render_template, request
from tensorflow.keras.models import load_model

lemmatizer = WordNetLemmatizer()
nltk.download('punkt')
nltk.download('wordnet')

intents = json.loads(open('intents.json').read())
words = pickle.load(open('texts.pkl','rb'))
classes = pickle.load(open('labels.pkl','rb'))
model = load_model('model.h5')

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    if ints:
        tag = ints[0]['intent']
        list_of_intents = intents_json['intents']
        for i in list_of_intents:
            if i['tag'] == tag:
                result = random.choice(i['responses'])
                break
        return result
    else:
        return "Sorry, I didn't understand that."

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    print(res)
    return res

# app = Flask(__name__,template_folder=r'C:\Users\HP\Downloads\Mental-health-Chatbot-master\Mental-health-Chatbot-master\templates')
app = Flask(__name__, template_folder='C:\\Users\\HP\\Downloads\\Mental-health-Chatbot-master\\Mental-health-Chatbot-master\\templates')

app.static_folder = 'C:\\Users\\HP\\Downloads\\Mental-health-Chatbot-master\\Mental-health-Chatbot-master\\static'


@app.route("/")
def home():
    return render_template("ui1.html")

@app.route("/bot")
def bot():
    return render_template("index.html")

@app.route("/get")
def get_bot_response():
    userText = request.args.get('msg')
    print(userText)
    response = chatbot_response(userText)
    print(response)
    return response

if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [23/Feb/2024 16:14:21] "GET /static/styles/ui1.css HTTP/1.1" 304 -
127.0.0.1 - - [23/Feb/2024 16:14:21] "GET /static/styles/Mental-health-Chatbot-master/static/img/ui.jpg HTTP/1.1" 404 -
127.0.0.1 - - [23/Feb/2024 16:14:24] "GET /static/img/pic.png HTTP/1.1" 304 -
127.0.0.1 - - [23/Feb/2024 16:15:08] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Feb/2024 16:15:08] "GET /static/styles/ui1.css HTTP/1.1" 200 -
127.0.0.1 - - [23/Feb/2024 16:15:09] "GET /static/styles/Mental-health-Chatbot-master/static/img/ui.jpg HTTP/1.1" 404 -
127.0.0.1 - - [23/Feb/2024 16:15:12] "GET /static/img/pic.png HTTP/1.1" 304 -
127.0.0.1 - - [23/Feb/2

In [ ]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               45568     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 129)               8385      
                                                                 
Total params: 62209 (243.00 KB)
Trainable params: 62209 (243.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
_____________

In [ ]:
def calculate_precision(test_dataset, predicted_intents):
    correct_predictions = 0
    total_predictions = len(predicted_intents)

    for i in range(total_predictions):
        if predicted_intents[i] == test_dataset[i]['intent']:
            correct_predictions += 1

    precision = correct_predictions / total_predictions
    return precision

# Assuming test_dataset contains test queries with corresponding intents
# and predicted_intents contains the intents predicted by the chatbot
precision = calculate_precision(test_dataset, predicted_intents)
print("Precision:", precision)


NameError: name 'test_dataset' is not defined

NameError: name 'test_dataset' is not defined

In [17]:
import nltk
nltk.download('popular')#  Natural Language Toolkit (NLTK) library installed will download and
# install a collection of popular NLTK data packages, including corpora,
# models etc
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np
from keras.models import load_model
model = load_model('/content/drive/MyDrive/Decobot/model.h5')
import json
import random




from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import spacy
from spacy.language import Language
from spacy_langdetect import LanguageDetector


# translator pipeline for english to swahili translations
# eng_swa_model_checkpoint = "Helsinki-NLP/opus-mt-en-swc"
# eng_swa_tokenizer = AutoTokenizer.from_pretrained("./model/en_sw/")
# eng_swa_model = AutoModelForSeq2SeqLM.from_pretrained("./model/en_sw/")

# eng_swa_translator = pipeline(
#     "text2text-generation",
#     model=eng_swa_model,
#     tokenizer=eng_swa_tokenizer,
# )

# def translate_text_eng_swa(text):
#     translated_text = eng_swa_translator(text, max_length=128, num_beams=5)[0]['generated_text']
#     return translated_text

# translator pipeline for swahili to english translations
# swa_eng_model_checkpoint = "Helsinki-NLP/opus-mt-swc-en"
# swa_eng_tokenizer = AutoTokenizer.from_pretrained("./model/sw_en/")
# swa_eng_model = AutoModelForSeq2SeqLM.from_pretrained("./model/sw_en/")

# swa_eng_translator = pipeline(
#     "text2text-generation",
#     model=swa_eng_model,
#     tokenizer=swa_eng_tokenizer,
# )

# def translate_text_swa_eng(text):
#     translated_text = swa_eng_translator(text, max_length=128, num_beams=5)[0]['generated_text']
#     return translated_text


# # Language detector
# def get_lang_detector(nlp, name):
#     return LanguageDetector()

# # # Load the English language model
nlp = spacy.load("en_core_web_sm")

# # Register the language detection factory
# Language.factory("language_detector", func=get_lang_detector)

# # # Add the language detection component to the pipeline
# nlp.add_pipe('language_detector', last=True)





intents = json.loads(open('intents.json').read())
words = pickle.load(open('texts.pkl','rb'))
classes = pickle.load(open('labels.pkl','rb'))
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    if ints:
        tag = ints[0]['intent']
        list_of_intents = intents_json['intents']
        for i in list_of_intents:
            if i['tag'] == tag:
                result = random.choice(i['responses'])
                break
        return result
    else:
        return "Sorry, I didn't understand that."

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    print(res)

    # # language detection
    # doc = nlp(res)
    # detected_language = doc._.language['language']
    # print(f"Detected language: {detected_language}")

    # translated_response = res

    # if detected_language == "en":
    #     pass
    # elif detected_language == 'sw':
    #     translated_response = translate_text_eng_swa(res)

    # print(translated_response)
    # return translated_response

# Creating GUI with flask
from flask import Flask, render_template, request
app = Flask(__name__,template_folder='/content/drive/MyDrive/Decobot/templates')
app.static_folder = 'static'
@app.route("/")
def home():
    return render_template("index.html")
@app.route("/get")
def get_bot_response():
    userText = request.args.get('msg')
    print(userText)

    # language detection
    # doc = nlp(userText)
    # detected_language = doc._.language['language']
    # print(f"Detected language: {detected_language}")

    # if detected_language == "en":
    response = chatbot_response(userText)
    # elif detected_language == 'sw':
    #     response = translate_text_eng_swa(chatbot_response(userText))

    print(response)
    return response
if __name__ == "__main__":
    app.run()

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\HP\AppData\Roaming\nltk_data...


KeyboardInterrupt: 

In [ ]:
import json
import random

# Load the JSON dataset
with open('intents.json', 'r') as file:
    dataset = json.load(file)

# Convert the dictionary to a list of tuples (query, intent)
dataset_list = [(sample['query'], sample['intent']) for sample in dataset]

# Shuffle the dataset
random.shuffle(dataset_list)

# Define the ratio for splitting the dataset
train_ratio = 0.8  # 80% for training, 20% for testing

# Split the dataset into training and testing parts
train_size = int(len(dataset_list) * train_ratio)
train_dataset = dataset_list[:train_size]
test_dataset = dataset_list[train_size:]

# Save or use the training and testing datasets as needed
# For example:
with open('train_dataset.json', 'w') as file:
    json.dump(train_dataset, file)

with open('test_dataset.json', 'w') as file:
    json.dump(test_dataset, file)
